# Topic Map

In [8]:
import os
import pandas as pd
import re
from nltk import sent_tokenize, word_tokenize
nltk.download('punkt')

transcription_dir_location_en = '../data/processed/transcription/en/'
docs = []

def parse_text(file_location): 
    transcriptions_df = pd.read_json(file_location)
    transcription_text = transcriptions_df[0].iloc[0]
    sentence_list = sent_tokenize(transcription_text)
    sentences = []
    for sentence in sentence_list:   
        docs.append(sentence)
        # sentences.append(sentence)
    # docs.append(' '.join(sentences))
    
for file_name in os.listdir(transcription_dir_location_en):
    parse_text(transcription_dir_location_en + file_name)

docs = list(filter(None, docs))
docs

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/frasercrichton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Another important chapter of our work in recent days concerns healthcare.',
 'The government has allocated over one billion euros for Regions and autonomous provinces, to limit the impact of payback in medical devices, on companies in the sector and to strengthen emergency rooms.',
 'Because the scenes of degradation, abandonment, confusion that we sometimes see in a difficult moment in our lives, that is, when we need emergency healthcare, are not tolerable.',
 'So we decided to increase the salaries and overtime resources of emergency room doctors.',
 'We have decided to limit the possibility that those who work in the public sector can easily and simultaneously also do so in the private sector, the so-called paid doctors.',
 'And we have decided to introduce harsher penalties for those who attack healthcare workers.',
 'Because it has also become intolerable that workers who do delicate and difficult work to save the lives of other people also have to fear for their own safety.',


In [16]:
import os
import pandas as pd
import re
import nltk
from nltk import sent_tokenize, word_tokenize

import openai
from bertopic.representation import OpenAI
from bertopic import BERTopic

openai.api_key=os.environ['openai_api_key'] 

# representation_model = OpenAI(delay_in_seconds=5)

def comp(PROMPT, MaxToken=50, outputs=3):
    # using OpenAI's Completion module that helps execute 
    # any tasks involving text 
    response = openai.Completion.create(
        # model name used here is text-davinci-003
        # there are many other models available under the 
        # umbrella of GPT-3
        model="text-davinci-003",
        # passing the user input 
        prompt=PROMPT,
        # generated output can have "max_tokens" number of tokens 
        max_tokens=MaxToken,
        # number of outputs generated in one call
        n=outputs
    )
    # creating a list to store all the outputs
    output = list()

    for k in response['choices']:
        output.append(k['text'].strip())
    return output


topic_model = BERTopic(representation_model=representation_model)
topics, probs = topic_model.fit_transform(docs)

prompt = """
I have topic that contains the following documents: \n[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short topic label in the following format:
topic: <topic label>
"""

representation_model = OpenAI(model="gpt-3.5-turbo", delay_in_seconds=10, chat=True, prompt=prompt)


comp(representation_model, MaxToken=3000, outputs=1)
# topic_model.get_topic_info()
# topic_model.get_topic_info()


TypeError: Object of type OpenAI is not JSON serializable

In [13]:
summarization_prompt = """
I have a topic that is described by the following keywords: [KEYWORDS]
In this topic, the following documents are a small but representative subset of all documents in the topic:
[DOCUMENTS]

Based on the information above, please give a description of this topic in the following format:
topic: <description>
"""

representation_model = OpenAI(model="gpt-3.5-turbo", chat=True, prompt=summarization_prompt, nr_docs=5, delay_in_seconds=3)

representation_model

OpenAI(chat=True, delay_in_seconds=3, model='gpt-3.5-turbo', nr_docs=5,
       prompt='\n'
              'I have a topic that is described by the following keywords: '
              '[KEYWORDS]\n'
              'In this topic, the following documents are a small but '
              'representative subset of all documents in the topic:\n'
              '[DOCUMENTS]\n'
              '\n'
              'Based on the information above, please give a description of '
              'this topic in the following format:\n'
              'topic: <description>\n')